In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from TumorTruths import TumorTruths
from sklearn.metrics import f1_score

In [2]:
svnFiles = ['SNVCalls_IS1.txt', 'SNVCalls_IS2.txt', 'SNVCalls_IS3.txt', 'SNVCalls_IS4.txt']
truthFiles = ['synthetic.challenge.set1.tumor.all.truth.vcf', 'synthetic.challenge.set2.tumor.all.truth.vcf', 'synthetic.challenge.set3.tumor.20pctmasked.truth.vcf', 'synthetic.challenge.set4.tumour.25pctmasked.truth.vcf']

In [3]:
def calculateF1ScoresByPercentile(svnFileName, truthFileName):
    svnCalls = pd.read_csv(svnFileName, delimiter='\t', dtype={'CHROM':pd.np.str})
    truths = TumorTruths(truthFileName)
    resultsArray = truths.GetTruths(svnCalls)
    svnCalls = svnCalls[svnCalls.columns[3:-1]]

    numPositivePredictions = svnCalls.sum(axis=1)
    scoresByPercentile = np.zeros(100)

    for percentile in range(100):
        fractionalPercentile = percentile / 100
        transformToPredictions = np.vectorize(lambda x: 1 if x >= (fractionalPercentile * len(svnCalls.columns)) else 0) 
        result = transformToPredictions(numPositivePredictions)
        scoresByPercentile[percentile] = f1_score(result, resultsArray)    
        
    return scoresByPercentile

In [5]:
scoresByPercentile = np.zeros((len(svnFiles), 100))
for k in range(len(svnFiles)):
    scoresByPercentile[k] = calculateF1ScoresByPercentile(svnFiles[k], truthFiles[k])
#     trainSvnFiles = svnFiles[:k] + svnFiles[k+1:]
#     trainTruthFiles = truthFiles[:k] + truthFiles[k+1:]
#     scoresByPercentile = np.zeros(100)

#     for i in range(len(trainSvnFiles)):  
#         scoresByPercentile += calculateF1ScoresByPercentile(trainSvnFiles[i], trainTruthFiles[i])

#     print np.max(scoresByPercentile)
#     print np.argmax(scoresByPercentile)
#     break

c:\Python27\lib\site-packages\sklearn\metrics\classification.py:960: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [11]:
from scipy import stats
zScores = stats.zscore(scoresByPercentile, axis=1)
medians = np.median(zScores, axis=0)

In [14]:
print np.argmax(medians)

30


In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(100), result)
plt.show()